In [1]:
import math
import pandas as pd


class Node:
    def __init__(self, feature=None, label=None):
        self.feature = feature  # feature to split on
        self.label = label      # label if it's a leaf node
        self.children = {}      # dict: feature_value -> child_node


def entropy(data, target_attr):
    values = data[target_attr].value_counts(normalize=True)
    return -sum(p * math.log2(p) for p in values)


def information_gain(data, feature, target_attr):
    total_entropy = entropy(data, target_attr)
    values = data[feature].unique()
    weighted_entropy = 0

    for value in values:
        subset = data[data[feature] == value]
        weighted_entropy += (len(subset) / len(data)) * entropy(subset, target_attr)

    return total_entropy - weighted_entropy


def id3(data, features, target_attr):
    labels = data[target_attr].unique()

    # Case 1: All examples have same label
    if len(labels) == 1:
        return Node(label=labels[0])

    # Case 2: No more features, return majority label
    if not features:
        majority_label = data[target_attr].mode()[0]
        return Node(label=majority_label)

    # Select best feature
    gains = {feature: information_gain(data, feature, target_attr) for feature in features}
    best_feature = max(gains, key=gains.get)

    root = Node(feature=best_feature)

    for value in data[best_feature].unique():
        subset = data[data[best_feature] == value]
        if subset.empty:
            majority_label = data[target_attr].mode()[0]
            child = Node(label=majority_label)
        else:
            child = id3(subset, [f for f in features if f != best_feature], target_attr)
        root.children[value] = child

    return root


def predict(node, sample):
    # Leaf node
    if node.label is not None:
        return node.label
    # Internal node
    feature_value = sample.get(node.feature)
    child = node.children.get(feature_value)
    if child is None:
        return None
    return predict(child, sample)


def print_tree(node, depth=0):
    if node.label is not None:
        print("  " * depth + f"Leaf: {node.label}")
    else:
        print("  " * depth + f"[{node.feature}]")
        for value, child in node.children.items():
            print("  " * (depth + 1) + f"{value} →")
            print_tree(child, depth + 2)


# Example usage
if __name__ == "__main__":
    data = pd.DataFrame([
        {"Outlook": "Sunny", "Temp": "Hot", "Humidity": "High", "Wind": "Weak", "Play": "No"},
        {"Outlook": "Sunny", "Temp": "Hot", "Humidity": "High", "Wind": "Strong", "Play": "No"},
        {"Outlook": "Overcast", "Temp": "Hot", "Humidity": "High", "Wind": "Weak", "Play": "Yes"},
        {"Outlook": "Rain", "Temp": "Mild", "Humidity": "High", "Wind": "Weak", "Play": "Yes"},
        {"Outlook": "Rain", "Temp": "Cool", "Humidity": "Normal", "Wind": "Weak", "Play": "Yes"},
        {"Outlook": "Rain", "Temp": "Cool", "Humidity": "Normal", "Wind": "Strong", "Play": "No"},
        {"Outlook": "Overcast", "Temp": "Cool", "Humidity": "Normal", "Wind": "Strong", "Play": "Yes"},
    ])

    features = ["Outlook", "Temp", "Humidity", "Wind"]
    target = "Play"

    tree = id3(data, features, target)

    print("Decision Tree:")
    print_tree(tree)

    test_sample = {"Outlook": "Rain", "Temp": "Cool", "Humidity": "High", "Wind": "Strong"}
    print("\nPrediction for test sample:", predict(tree, test_sample))


Decision Tree:
[Outlook]
  Sunny →
    Leaf: No
  Overcast →
    Leaf: Yes
  Rain →
    [Wind]
      Weak →
        Leaf: Yes
      Strong →
        Leaf: No

Prediction for test sample: No
